# Módulo: MODELOS DE INTELIGENCIA ARTIFICIAL
## Ejercicio práctico 1.
## Sistema Basado en Reglas   
### Análisis de la calidad del aire

De forma similar a lo realizado en el notebook SBR_IRIS, construye un sistema de clasificación de la calidad del aire (AirQuality - AQI) en función de la cantidad de Ozono detectado en el mismo.   
Los valores del nivel de AQI se muestran en la siguiente tabla:  


| NIVEL AQI | RANGO-DESCRIPCIÓN|
|:---------:|:----------------:|
|   BUENO   |     0 - 50       |
| MODERADO  |    51 - 100      |
|CONTAMINADO|   101 - 150      |
|  INSANO   |   151 - 200      |
| PELIGROSO |       > 200      |

Se pide intentar establecer la clasificación según la tabla anterior en base a alguna de las características que conforman el dataset (Valor de O2, Valor de CO, Valor de NO2, Valor de PM2.5, Longitud y Latitud).   
Como los datos son numéricos y no faltan valores, no se requiere ningún procesamiento previo.   

Nuestro objetivo es predecir el AQI, por lo que esta tarea sería de clasificación o de regresión, pero en este caso vamos a tratar de establecer un SBR basado en una de las variables independientes citadas.

<u>Nota sobre la característica ***PM2.5***</u> :   
Este valor se corresponde con la concentración del material particulado (*PM - Particulate Matter*) de la fracción de tamaño PM2,5 (2.5 micrometros), el cúal se encuentra íntimamente asociado a problemas de salud dado que entre sus efectos aparecen impactos claros sobre las enfermedades cardiacas y pulmonares.   

Importamos librerías necesarias

In [1]:
#codebase
##Lee el archivo y añade el codigo necesario en python para solucionar las cuestiones que indica
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Sistema Basado en Reglas para Análisis de Calidad del Aire (AQI)

Voy a crear un sistema basado en reglas para clasificar la calidad del aire según los niveles de la tabla proporcionada, utilizando una de las características disponibles en el dataset.



In [ ]:





# Exploramos correlaciones entre variables
plt.figure(figsize=(12, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación')
plt.show()

# Visualizamos la relación entre O3 y AQI
plt.figure(figsize=(10, 6))
plt.scatter(data['O3'], data['AQI_Value'], alpha=0.5)
plt.xlabel('Nivel de Ozono (O3)')
plt.ylabel('Valor AQI')
plt.title('Relación entre Ozono y AQI')
plt.grid(True)
plt.show()





# Veamos la distribución de categorías
print("\nDistribución de categorías AQI:")
print(data['AQI_Categoria'].value_counts())

# Visualizamos la distribución
plt.figure(figsize=(10, 6))
sns.countplot(x='AQI_Categoria', data=data, order=['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO', 'PELIGROSO'])
plt.title('Distribución de Categorías AQI')
plt.xlabel('Categoría')
plt.ylabel('Cantidad')
plt.grid(True, axis='y')
plt.show()

# Ahora implementamos un SBR basado en el nivel de Ozono
# Buscamos los límites óptimos para clasificar según O3

# Separamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    data[['O3']], data['AQI_Categoria'], test_size=0.3, random_state=42)

# Encontramos los umbrales óptimos para O3 basados en los datos de entrenamiento
umbrales = {}
for categoria in ['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO']:
    # Filtramos registros de la categoría actual
    cat_data = X_train[y_train == categoria]
    if not cat_data.empty:
        umbrales[categoria] = cat_data['O3'].max()

# Ordenamos umbrales
umbrales = dict(sorted(umbrales.items(), key=lambda x: x[1]))
print("\nUmbrales de O3 por categoría:")
print(umbrales)

# Creamos el sistema de reglas basado en O3
def clasificar_por_o3(o3_valor):
    if o3_valor <= umbrales.get('BUENO', 50):
        return "BUENO"
    elif o3_valor <= umbrales.get('MODERADO', 100):
        return "MODERADO"
    elif o3_valor <= umbrales.get('CONTAMINADO', 150):
        return "CONTAMINADO"
    elif o3_valor <= umbrales.get('INSANO', 200):
        return "INSANO"
    else:
        return "PELIGROSO"

# Evaluamos el SBR en los datos de prueba
X_test['Prediccion'] = X_test['O3'].apply(clasificar_por_o3)

# Calculamos precisión
accuracy = accuracy_score(y_test, X_test['Prediccion'])
print(f"\nPrecisión del SBR: {accuracy:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, X_test['Prediccion'], 
                      labels=['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO', 'PELIGROSO'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO', 'PELIGROSO'],
            yticklabels=['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO', 'PELIGROSO'])
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, X_test['Prediccion']))

# Visualización final: valores de O3 vs Categoría AQI
plt.figure(figsize=(12, 6))
sns.boxplot(x='AQI_Categoria', y='O3', data=data, 
           order=['BUENO', 'MODERADO', 'CONTAMINADO', 'INSANO', 'PELIGROSO'])
plt.title('Distribución de O3 por Categoría AQI')
plt.xlabel('Categoría AQI')
plt.ylabel('Nivel de Ozono (O3)')
plt.grid(True, axis='y')
plt.show()



Este código:

1. Genera un conjunto de datos sintético de calidad del aire (puedes reemplazar esto con la carga de tu dataset real)
2. Explora las correlaciones entre variables para ver qué característica se relaciona mejor con el AQI
3. Implementa un sistema basado en reglas usando los rangos de la tabla
4. Crea un segundo sistema que aprende umbrales óptimos de O3 a partir de los datos
5. Evalúa el rendimiento con métricas como precisión y matriz de confusión
6. Visualiza los resultados

Si tienes un conjunto de datos específico, simplemente reemplaza la parte de generación de datos con:



In [2]:
# Carga de tu dataset específico
data = pd.read_csv('ruta_a_tu_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'ruta_a_tu_dataset.csv'



Y ajusta los nombres de las columnas según sea necesario.

In [ ]:
#codebase

Cargamos dataset *airquality.csv*

In [3]:
# Cargamos el dataset
data = pd.read_csv('datasets/airquality.csv')

Mostramos los primeros registros

In [12]:
#codebase# Visualización de datos
print("Estadísticas descriptivas:")
print(data.describe())

Asignación (mapeo) del nivel de calidad según los valores del mismo

In [ ]:
#codebase


Análisis breve del dataset (EDA)

In [ ]:
#codebase

Definición de reglas

In [ ]:
#codebase
# Implementamos el Sistema Basado en Reglas
def clasificar_aqi(valor):
    if valor <= 50:
        return "BUENO"
    elif valor <= 100:
        return "MODERADO"
    elif valor <= 150:
        return "CONTAMINADO"
    elif valor <= 200:
        return "INSANO"
    else:
        return "PELIGROSO"

Aplicación de reglas

In [ ]:
#codebase
# Aplicamos la clasificación
data['AQI_Categoria'] = data['AQI_Value'].apply(clasificar_aqi)

Visualización gráfica

In [ ]:
#codebase

In [ ]:
#codebase